Imports

In [2]:
from torch.utils.data import Dataset
from os import listdir
from os.path import isfile, join
from matplotlib import pyplot as plt
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

import cv2
import json
import numpy as np
from tqdm import tqdm


Custom dataset

In [4]:
class SegmentationDataset(Dataset):
	def __init__(self, imagePath, maskPath, transforms=None):
		# store the image and mask filepaths, and augmentation
		# transforms
		self.imageFiles = [f for f in listdir(imagePath) if isfile(join(imagePath, f))]
		self.imagePath = imagePath
		f = open(maskPath)
		self.maskJSON = json.load(f)
		self.transforms = transforms
	def __len__(self):
		# return the number of total samples contained in the dataset
		return len(self.imageFiles)
	def __getitem__(self, idx):				#IMPLEMENT
		# grab the image path from the current index
		imagePath = join(self.imagePath,self.imageFiles[idx])
		# load the image from disk, swap its channels from BGR to RGB,
		# and read the associated mask from disk in grayscale mode
		image = cv2.imread(imagePath)
		image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
		#Creating the mask
		points = []
		for point in self.maskJSON[self.imageFiles[idx]]['bounds_x_y']:
			points.append((point['x'],point['y']))
		polygon = Polygon(points)
		mask = np.zeros((512,512))
		for y in range(512):
			for x in range(512):
				if polygon.contains(Point(x,y)):
					mask[x,y]=1
		return (image, mask, self.imageFiles[idx])

In [4]:
def displayImageAndMask(image,mask):
    fig = plt.figure(figsize=(10, 7))
    fig.add_subplot(1,2,1)
    plt.imshow(mask,interpolation='nearest')
    plt.show()
    fig.add_subplot(1,2,2)
    plt.imshow(mask,interpolation='nearest')
    plt.show()

def saveMask(mask,fileName):
    fileOut = 'masks/'+fileName[:-4]+'.out'
    with open(fileOut, "wb") as f:
        np.savetxt(f, mask.astype(int), fmt='%i', delimiter=",")

In [3]:
from tqdm import tqdm

data = SegmentationDataset('images','metadata.json')
for i in tqdm(range(len(data))):
    _,mask,fileName = data[i]
    saveMask(mask,fileName)



NameError: name 'SegmentationDataset' is not defined

In [9]:
fJSON = open('metadata.json')
maskJSON = json.load(fJSON)
fileNames = list(maskJSON.keys())

print(len(maskJSON[list(maskJSON.keys())[243]]['bounds_lat_lon']))
print(maskJSON[list(maskJSON.keys())[243]]['bounds_lat_lon'])

imageFiles = [f for f in listdir('images') if isfile(join('images', f))]


# c=0
# for i in tqdm(range(len(fileNames))):
#     if fileNames[i] in imageFiles:
#         c += 1

# print(c,'/',len(fileNames))



5
[{'lat': 43.7491912, 'lon': -1.3265787}, {'lat': 43.7491306, 'lon': -1.3265054}, {'lat': 43.7491659, 'lon': -1.326452}, {'lat': 43.7492215, 'lon': -1.3265289}, {'lat': 43.7491912, 'lon': -1.3265787}]


In [36]:
fJSON = open('metadata.json')
maskJSON = json.load(fJSON)
fileNames = list(maskJSON.keys())
imageFiles = [f for f in listdir('images') if isfile(join('images', f))]

createdFiles = [f for f in listdir('masks') if isfile(join('masks', f))]

for i in tqdm(range(len(fileNames))):
    fileOut = fileNames[i][:-4]+'.out'
    if (fileOut not in createdFiles) and (fileNames[i] in imageFiles):
        points = []
        for point in maskJSON[fileNames[i]]['bounds_x_y']:
            points.append((point['x'],point['y']))
        polygon = Polygon(points)
        mask = np.zeros((512,512))
        for y in range(512):
            for x in range(512):
                if polygon.contains(Point(x,y)):
                    mask[y,x]=1
        saveMask(mask,fileNames[i])

  1%|          | 77/14964 [01:11<16:30:26,  3.99s/it]Exception ignored in: <function BaseGeometry.__del__ at 0x000001BD8C401E50>
Traceback (most recent call last):
  File "C:\Users\weeck\AppData\Local\Programs\Python\Python39\lib\site-packages\shapely\geometry\base.py", line 209, in __del__
    self._empty(val=None)
  File "C:\Users\weeck\AppData\Local\Programs\Python\Python39\lib\site-packages\shapely\geometry\base.py", line 194, in _empty
    self._lgeos.GEOSGeom_destroy(self.__geom__)
KeyboardInterrupt: 
  1%|          | 80/14964 [01:25<18:26:51,  4.46s/it]

In [15]:
i = 400
createdFiles = []
fileOut = fileNames[i][:-4]+'.out'
if fileOut not in createdFiles:
    points = []
    for point in maskJSON[fileNames[i]]['bounds_x_y']:
        points.append((point['x'],point['y']))
    polygon = Polygon(points)
    mask = np.zeros((512,512))
    for y in range(512):
        for x in range(512):
            if polygon.contains(Point(x,y)):
                mask[y,x]=1
    saveMask(mask,fileNames[i])